<a href="https://colab.research.google.com/github/AMS-L/NLP-LAB7/blob/main/nlp_lab7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NLP - LAB 7


In [3]:
!pip install transformers
!pip install datasets
!pip install transformers==4.28.0
!pip install --upgrade accelerate


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
import transformers
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import torch
from transformers import DataCollatorWithPadding

## Sentence Classification

### 1. Fine-tune the model on the training data. 

In [5]:
dataset = load_dataset("imdb")
train_dataset, test_dataset = dataset['train'], dataset['test']

  0%|          | 0/3 [00:00<?, ?it/s]

In [6]:
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize(batch):
    return tokenizer(batch['text'], padding=True, truncation=True)
    
train_dataset, test_dataset = dataset['train'], dataset['test']
train_dataset = train_dataset.map(tokenize, batched=True, batch_size=len(train_dataset))
test_dataset = test_dataset.map(tokenize, batched=True, batch_size=len(test_dataset))

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 7>:7                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'batch' is not defined

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
# Training arguments
args = TrainingArguments(
    "test-imdb",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
)


In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.weight', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.weight', 'classifier

In [ ]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).astype(np.float32).mean().item()}


In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)


trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.223000,0.193714,0.927120


TrainOutput(global_step=1563, training_loss=0.2581864001273499, metrics={'train_runtime': 1592.1889, 'train_samples_per_second': 15.702, 'train_steps_per_second': 0.982, 'total_flos': 3311684966400000.0, 'train_loss': 0.2581864001273499, 'epoch': 1.0})

### 3. Evaluate the model in term of accuracy on the test data.

In [ ]:
trainer.evaluate()

{'eval_loss': 0.19371409714221954,
 'eval_accuracy': 0.9271199703216553,
 'eval_runtime': 427.7291,
 'eval_samples_per_second': 58.448,
 'eval_steps_per_second': 3.654,
 'epoch': 1.0}

On a ici une accuracy de 92,7 %.

In [ ]:
predictions = trainer.predict(test_dataset)
predicted_labels = np.argmax(predictions.predictions, axis=-1)

actual_labels = predictions.label_ids

 ### 4. For at least 2 samples which have been wrongly classified in the test set, try explaining why the model could have been wrong.

In [1]:
misclassified_indices = np.where(predicted_labels != actual_labels)[0]

for i in misclassified_indices[:10]:
    print(f"Text: {test_dataset['text'][i]}")
    print(f"Predicted label: {predicted_labels[i]}")
    print(f"Actual label: {actual_labels[i]}\n")

NameError: ignored

On peut voir dans ce contexte que les deux commentaires sont positifs mais qu'ils ressortent négatif dans la classification. Cela pourrait être du au fait que la logique de base de la transformer consiste à utiliser des mécanismes d'attention pour calculer des représentations contextuelles des mots ou des tokens dans une séquence. 
Ces représentations contextuelles capturent les informations contextuelles à la fois locales et globales, ce qui permet au modèle de comprendre les relations et les dépendances entre les éléments de la séquence. 

Dans ce contexte, on voit que le premier commentaire établit beaucoup de formulations contextuelles négatives, et une énumération des défauts du film cité ce qui peut porter à confusion le transformer.

Le deuxième admet beaucoup d'adjectifs négatifs mais la nuance entre la description négatives des personnage dans un contexte fictif et le commentaire sur le film en lui même peut induire le transformer en erreur.

### 5. What are the advantages and inconvenient of using this model in production compared to the naive Bayes we implemented in the first part of the course? And compared to a recurrent model like an RNN or an LSTM?

#### -> Les avantages :

  Le modèle de transformer comme DistilBERT est capable de capturer des relations complexes entre les mots ou les tokens d'une séquence grâce à son architecture d'attention. Contrairement à Naive Bayes, qui suppose une indépendance entre les caractéristiques, DistilBERT peut modéliser les dépendances à longue distance, ce qui lui permet de saisir des informations contextuelles plus riches et de mieux comprendre le langage naturel.

  Dans un second temps, DistilBERT apprend aussi des représentations de mots ou de tokens directement à partir des données, ce qui lui permet d'adapter ces représentations à la tâche spécifique. En revanche, Naive Bayes utilise des statistiques de fréquence pour estimer les probabilités, ce qui peut être moins adaptatif et moins précis dans des cas où les caractéristiques sont complexes et fortement interdépendantes.

  Pour finir, DistilBERT intègre une compréhension sémantique des mots ou des tokens grâce à l'utilisation d'embeddings, qui représentent les mots dans un espace vectoriel continu. Cela permet au modèle de comprendre les similitudes et les relations entre les mots, ce qui peut être utile pour des tâches telles que la recherche de similarités ou la génération de texte cohérent.

#### -> Les inconvénient :

  DistilBERT, est plus complexes et nécessite des ressources computationnelles plus importantes que les modèles Naive Bayes. Ils peuvent être plus lents à l'entraînement et à l'inférence, ce qui peut être un inconvénient dans des scénarios où le temps de réponse est important.

  Les modèles de transformer ont souvent besoin d'un volume plus important de données d'entraînement pour atteindre de bonnes performances. Par conséquent, ils peuvent nécessiter un ensemble de données plus large et plus diversifié par rapport à Naive Bayes, qui est moins exigeant en termes de données d'entraînement.

#### -> Comparaison avec les modèles récurrents (RNN ou LSTM) :

  Les modèles récurrents sont conçus pour capturer les dépendances séquentielles à long terme dans une séquence. Cela peut être avantageux pour les tâches où l'ordre des éléments est important, comme la traduction automatique ou la génération de texte cohérent. DistilBERT, en revanche, n'a pas d'architecture récurrente et dépend entièrement des mécanismes d'attention pour capturer les dépendances.

  Par la suite, DistilBERT, peut prendre en compte le contexte global de la séquence grâce à l'attention sur tous les tokens. Cela lui permet de saisir les relations non seulement à l'intérieur d'une fenêtre de contexte fixe, mais aussi à travers toute la séquence. Les modèles récurrents ont quant à eux une mémoire limitée et peuvent avoir du mal à capter les dépendances à très long terme.
  
  Pour finir, DistilBERT bénéficie  d'un préentraînement sur de vastes ensembles de données non supervisées, ce qui lui permet d'apprendre des représentations linguistiques générales. 
  Cela peut être bénéfique pour des tâches spécifiques avec des ensembles de données plus petits. Les modèles récurrents, en revanche, nécessitent généralement un entraînement plus spécifique à la tâche.

### 6. The model only accepts inputs of maximum 512 tokens. Propose and implement a solution that goes around that. 